### Training a DNN

#### References:
- [Hands-on Machine Learning with Scikit-Learn, Keras and TensorFlow](https://www.amazon.com/Hands-Machine-Learning-Scikit-Learn-TensorFlow/dp/1492032646)

<a href="https://colab.research.google.com/github/paulaceccon/dl-from-scratch/blob/main/notebooks/dnn_tensorflow.ipynb" target="_parent" style="float: left;"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>